# Study of Finite Volume Schemes for the 2D Diffusion Equation

**Author:** Othmane OUKBIL



### Abstract
This report presents the study and comparison of two finite volume schemes for approximating the solution of a 2D diffusion equation. We describe the mathematical model, the domain meshing, the finite volume schemes used, and the approximation of diffusion fluxes on the edges.

### Introduction
The 2D diffusion equation studied in this report is of the following form:
$$
\frac{\partial u}{\partial t} - D \Delta u = 0
$$
where $D$ is the diffusion coefficient and $u = u(x, y, t)$ denotes the temperature or concentration at a point $(x, y)$ at time $t$.

### Meshing of $\Omega$
The domain $\Omega$ is discretized using a finite volume admissible mesh, with Delaunay triangles. The control volumes are the triangles $K$ of the mesh. We denote:
- $x_K$: the orthocenter of the triangle $K$,
- $|K|$: the area of the triangle $K$,
- $|e|$: the length of the edge $e$ of $K$,
- $\varepsilon_K$: the set of edges $e$ of $K$.

Time is discretized into steps $\Delta t$, and $u^n_K$ represents the approximate value of $u$ on the control volume $K$ at time $t^n = n \Delta t$.


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 21/06/2024
@author: OUKBIL Othmane
"""

from mpi4py import MPI
import timeit

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

from manapy.ddm import readmesh
from manapy.ddm import Domain

from manapy.tools.pyccel_tools import initialisation_gaussian_2d, update_new_value, time_step
                          
from manapy.fvm.pyccel_fvm import (explicitscheme_convective_2d,
                                   explicitscheme_dissipative)

#from manapy.ast import Variable, LinearSystem

import numpy as np
import os

start = timeit.default_timer()

# mesh directory

dim = 2
readmesh('/home/othmane/Downloads/tresfin.msh', dim=dim, periodic=[0,0,0])

#Create the informations about cells, faces and nodes
domain = Domain(dim=dim)

faces = domain.faces
cells = domain.cells
halos = domain.halos
nodes = domain.nodes

nbnodes = domain.nbnodes
nbfaces = domain.nbfaces
nbcells = domain.nbcells


Starting ....
Number of Cells :  10484
Number of Nodes :  5377


In [4]:
import numpy.linalg as LA

def circumcenter(C):
    ax = C[0][0]
    ay = C[0][1]
    bx = C[1][0]
    by = C[1][1]
    cx = C[2][0]
    cy = C[2][1]
    d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
    ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
    uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d
    return (ux, uy)

def dist(x, y):
    x=np.asarray(x)
    y=np.asarray(y)
    return LA.norm(x - y)

In [5]:
def cell_ver_coor():
    Cells_Cord=[]
    for s in cells._nodeid :
        Cell_Cord =[]
        for i in s :
            Cell_Cord.append(nodes._vertex[i][:-2])
        Cells_Cord.append(Cell_Cord)   
    return Cells_Cord

Cells_Cord = cell_ver_coor()

## Test 1: Diffusion of a Gaussian in a Square

#### Initial Condition
We choose an initial condition in the form of a Gaussian:
$$
u(x, y, 0) = u_0(x, y) = u_{\max} \exp \left( -\frac{x^2 + y^2}{\sigma^2} \right)
$$
where the maximum amplitude is $ u_{\max} = 1 $ and the constant $ \sigma = 0.25 $.

#### Boundary Conditions
We use fictitious cells to handle Dirichlet boundary conditions at the edges.
#### Visualization
We use PARAVIEW to visualize the isovalues and the numerical results.



In [7]:
 def save_paraview_one_variable(w, cells, nodes, dim, name):
    
    if dim == 2:
        elements = {"triangle": cells}
    elif dim == 3:
        elements = {"tetra": cells}

    points = []
    for i in nodes:
        points.append([i[0], i[1], i[2]])
    
    cells  = np.array(cells)
    points = np.array(points)
    
    data = {"w" : w}
   
    if len(w) == len(cells):
        data = {"w": data}

    if len(w) == len(cells):
        meshio.write_points_cells("visu"+name+".vtu",
                                  points, elements, cell_data=data, file_format="vtu")
    else:
        meshio.write_points_cells("visu"+str(w)+".vtu",
                                  points, elements, point_data=data, file_format="vtu")


In [8]:
import numpy as np
import meshio
def Sol_init(x, y, alpha = 4):
    return np.exp(-1*(x**2+y**2)/0.25**2)


u_init = [ Sol_init(cell[0], cell[1]) for cell in domain.cells.center]
u_init = np.array(u_init)

save_n=0
save_paraview_one_variable(u_init, cells._nodeid, nodes._vertex, 2, "res_VF4_trfin_"+str(save_n))

In [3]:
#Cells Centre
Cells_centre= [cell[:-1] for cell in cells._center]

### Finite Volume Scheme
The finite volume formulation for equation (1) is obtained by integrating the PDE over each control volume $K$. Using the divergence theorem, we get:
$$
|K| \frac{\partial u_K}{\partial t} + \sum_{e_i \in \varepsilon_K} F_{K,e_i} = 0
$$
where $F_{K,e} = -D \int_e \nabla u \cdot \mathbf{n}_{K,e} \, d\sigma$ is the exact diffusion flux on the edge $e$ of $K$ and $\mathbf{n}_{K,e}$ is the unit normal vector pointing outwards from the triangle $K$.

Using an explicit Euler scheme, the iterative form of the finite volume scheme is given by:
$$
u^{n+1}_K = u^n_K - \frac{\Delta t}{|K|} \sum_{e_i \in \varepsilon_K} F^n_{K,e_i}
$$

### Approximation of Diffusion Fluxes on Edges

#### Scheme 1: (VF4)
The gradient in the normal direction to the edge \(e_i\) is approximated by:
$$
\int_{e_i} \nabla u \cdot \mathbf{n}_{K,e_i} \, d\sigma \approx |e_i| \frac{u_{L_i} - u_K}{d_{K,L_i}}
$$
where $K$ and $L_i$ are the two triangles separated by the edge $e_i$ and $d_{K,L_i} = \text{dist}(x_K, x_{L_i}).$


In [6]:
#Orthocentre
Orthocentre = []
for i in range(nbcells):
    Orthocentre.append(circumcenter(Cells_Cord[i]))

In [21]:
#dt
temps=0
T=0.15
mx=-1
for i in range(nbcells):
    somme=0
    for j in range(len(cells.faceid[i])):
        face=cells.faceid[i][j]
        k = faces.cellid[face][0]
        l = faces.cellid[face][1]
        dst= dist(Orthocentre[k], Orthocentre[l])
        mesure = faces.mesure[face]
        somme+=mesure/dst
    K=cells.volume[i]
    somme=somme/K
    mx=max(somme,mx)
    
lmbda=0.8
D=1
dt=lmbda/(D*mx)  

print(dt)

0.00010123983974205996


Show that the explicit VF scheme 

$$ u^{n+1}_K = u^n_K - \frac{\Delta t}{|K|} \sum_{e_i \in \varepsilon_K} F^n_{K,e_i} $$

using the VF4 approximation for diffusion fluxes verifies a discrete maximum principle and is $L^\infty$-stable under the condition:

$$
\lambda = \max_{K \in \mathcal{T}} \left( \frac{D \Delta t}{|K|} \sum_{e_{i} \in  \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}} \right) \leq 1
$$

#### Maximum Principle

The explicit Euler scheme is given by:

$$ u_K^{n+1} = u_K^n - \frac{\Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} F_{K,e_{i}}^n $$

where the VF4 scheme is given by:

$$(VF4) \quad F_{K,e_{i}}^n = -D|e_{i}| \frac{u_{L_{i}} - u_K}{d_{K,L_{i}}} $$

Suppose $u_K^0 \geq 0$ and $u^n \geq 0$ over $\Omega$. Then $\forall K \subset \Omega, \, u_K^n \geq 0$. 

Let's show that $u_K^{n+1} \geq 0$:

$$ u_K^{n+1} = \left(1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}}\right) u_K^n + \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} |e_{i}| \frac{u_{L_{i}}^n}{d_{K,L_{i}}} $$

$$ = \left(1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}}\right) u_K^n + \frac{D \Delta t}{|K|} \sum_{e_{i} \epsilon_{K}} |e_{i}|\frac{u_{L_{i}}^n}{d_{K,L_{i}}} $$

For $u_K^{n+1} \geq 0$, it suffices that:

$$ 1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}} \geq 0 \quad \forall K $$

And since

$$ \lambda = \max_{K \in \mathcal{T}} \left( \frac{D \Delta t}{|K|} \sum_{e_{i} \in  \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}} \right) \leq 1 $$

Thus, $u_K^{n+1} \geq 0 , \forall K$, which implies $u_K^{n} \geq 0 , \forall K$.

#### Stability

\begin{align*}
|u_{K}^{n+1}| &= \left| \left(1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}}\right) u_K^n + \frac{D \Delta t}{|K|} \sum_{e_{i} \epsilon_{K}} |e_{i}|\frac{u_{L_{i}}^n}{d_{K,L_{i}}} \right|\\\\
&\leq \left(1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}}\right) |u_K^n| + \frac{D \Delta t}{|K|} \sum_{e_{i} \epsilon_{K}} |e_{i}|\frac{|u_{L_{i}}^n|}{d_{K,L_{i}}}\\\\
&\leq \left(1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}}\right) \|u^n\|_{\infty} + \frac{D \Delta t}{|K|} \sum_{e_{i} \epsilon_{K}} \frac{|e_{i}|}{d_{K,L_{i}}} \|u^n\|_{\infty}\\\\
&\leq \|u^n\|_{\infty}\\\\
\text{Thus} \quad \|u^{n+1}\|_{\infty} &\leq \|u^{n}\|_{\infty} \\\\
\text{This implies} \quad \|u^{n}\|_{\infty} &\leq \|u^{0}\|_{\infty}
\end{align*}

Therefore, the scheme is stable under the condition $\lambda \leq 1$.


In [11]:
temps=0
T=0.15

save_n = 0

u=np.copy(u_init)
i=0
unew = np.zeros(nbcells)
while temps<=T:
    temps+=dt
    save_n+=1
   
    for j in range(nbcells):
        list_faces = cells.faceid[j]
        elem = 0
        for k in range(3):
            mesure1 = faces.mesure[list_faces[k]]
            dist1 = dist(Orthocentre[faces.cellid[list_faces[k]][0]],Orthocentre[faces.cellid[list_faces[k]][1]])
            if j == faces.cellid[list_faces[k]][0]:
                u_diff = u[faces.cellid[list_faces[k]][1]] - u[faces.cellid[list_faces[k]][0]]
            else :
                u_diff = u[faces.cellid[list_faces[k]][0]] - u[faces.cellid[list_faces[k]][1]]
            elem += mesure1*u_diff/dist1
        unew[j] = u[j] - dt/(cells.volume[j]) * (-D)*elem

    u=np.copy(unew)
    i+=1
    save_paraview_one_variable(u, cells._nodeid, nodes._vertex, 2, "res_VF4_"+str(save_n))
    print(temps)

3.288644211872977e-05
6.577288423745955e-05
9.865932635618932e-05
0.0001315457684749191
0.00016443221059364886
0.00019731865271237862
0.0002302050948311084
0.0002630915369498382
0.000295977979068568
0.00032886442118729777
0.00036175086330602756
0.00039463730542475735
0.00042752374754348715
0.00046041018966221694
0.0004932966317809467
0.0005261830738996765
0.0005590695160184062
0.000591955958137136
0.0006248424002558657
0.0006577288423745954
0.0006906152844933252
0.0007235017266120549
0.0007563881687307846
0.0007892746108495144
0.0008221610529682441
0.0008550474950869739
0.0008879339372057036
0.0009208203793244333
0.0009537068214431631
0.0009865932635618928
0.0010194797056806227
0.0010523661477993525
0.0010852525899180824
0.0011181390320368122
0.001151025474155542
0.001183911916274272
0.0012167983583930017
0.0012496848005117316
0.0012825712426304614
0.0013154576847491913
0.0013483441268679211
0.001381230568986651
0.0014141170111053808
0.0014470034532241107
0.0014798898953428405
0.001512

0.012792825984185907
0.012825712426304637
0.012858598868423367
0.012891485310542097
0.012924371752660826
0.012957258194779556
0.012990144636898286
0.013023031079017016
0.013055917521135746
0.013088803963254476
0.013121690405373206
0.013154576847491935
0.013187463289610665
0.013220349731729395
0.013253236173848125
0.013286122615966855
0.013319009058085585
0.013351895500204315
0.013384781942323044
0.013417668384441774
0.013450554826560504
0.013483441268679234
0.013516327710797964
0.013549214152916694
0.013582100595035423
0.013614987037154153
0.013647873479272883
0.013680759921391613
0.013713646363510343
0.013746532805629073
0.013779419247747803
0.013812305689866532
0.013845192131985262
0.013878078574103992
0.013910965016222722
0.013943851458341452
0.013976737900460182
0.014009624342578911
0.014042510784697641
0.014075397226816371
0.014108283668935101
0.01414117011105383
0.01417405655317256
0.01420694299529129
0.01423982943741002
0.01427271587952875
0.01430560232164748
0.01433848876376621

0.025848743505321656
0.025881629947440386
0.025914516389559116
0.025947402831677846
0.025980289273796576
0.026013175715915306
0.026046062158034036
0.026078948600152765
0.026111835042271495
0.026144721484390225
0.026177607926508955
0.026210494368627685
0.026243380810746415
0.026276267252865144
0.026309153694983874
0.026342040137102604
0.026374926579221334
0.026407813021340064
0.026440699463458794
0.026473585905577524
0.026506472347696253
0.026539358789814983
0.026572245231933713
0.026605131674052443
0.026638018116171173
0.026670904558289903
0.026703791000408632
0.026736677442527362
0.026769563884646092
0.026802450326764822
0.026835336768883552
0.02686822321100228
0.02690110965312101
0.02693399609523974
0.02696688253735847
0.0269997689794772
0.02703265542159593
0.02706554186371466
0.02709842830583339
0.02713131474795212
0.02716420119007085
0.02719708763218958
0.02722997407430831
0.02726286051642704
0.02729574695854577
0.0273286334006645
0.02736151984278323
0.02739440628490196
0.027427292

0.03906909323705023
0.03910197967916896
0.039134866121287686
0.03916775256340641
0.03920063900552514
0.039233525447643865
0.03926641188976259
0.03929929833188132
0.039332184774000044
0.03936507121611877
0.0393979576582375
0.03943084410035622
0.03946373054247495
0.039496616984593676
0.0395295034267124
0.03956238986883113
0.039595276310949855
0.03962816275306858
0.03966104919518731
0.039693935637306034
0.03972682207942476
0.03975970852154349
0.03979259496366221
0.03982548140578094
0.039858367847899666
0.03989125429001839
0.03992414073213712
0.039957027174255845
0.03998991361637457
0.0400228000584933
0.040055686500612024
0.04008857294273075
0.04012145938484948
0.0401543458269682
0.04018723226908693
0.040220118711205656
0.04025300515332438
0.04028589159544311
0.040318778037561835
0.04035166447968056
0.04038455092179929
0.040417437363918014
0.04045032380603674
0.04048321024815547
0.04051609669027419
0.04054898313239292
0.040581869574511646
0.04061475601663037
0.0406476424587491
0.0406805289

0.052388102295134416
0.05242098873725314
0.05245387517937187
0.052486761621490595
0.05251964806360932
0.05255253450572805
0.052585420947846774
0.0526183073899655
0.05265119383208423
0.05268408027420295
0.05271696671632168
0.052749853158440406
0.05278273960055913
0.05281562604267786
0.052848512484796585
0.05288139892691531
0.05291428536903404
0.052947171811152764
0.05298005825327149
0.05301294469539022
0.05304583113750894
0.05307871757962767
0.053111604021746396
0.05314449046386512
0.05317737690598385
0.053210263348102575
0.0532431497902213
0.05327603623234003
0.053308922674458754
0.05334180911657748
0.05337469555869621
0.053407582000814934
0.05344046844293366
0.053473354885052386
0.05350624132717111
0.05353912776928984
0.053572014211408565
0.05360490065352729
0.05363778709564602
0.053670673537764745
0.05370355997988347
0.0537364464220022
0.053769332864120924
0.05380221930623965
0.053835105748358376
0.0538679921904771
0.05390087863259583
0.053933765074714556
0.05396665151683328
0.053999

...

0.09293708542752405
0.09296997186964277
0.0930028583117615
0.09303574475388023
0.09306863119599895
0.09310151763811768
0.0931344040802364
0.09316729052235513
0.09320017696447386
0.09323306340659258
0.09326594984871131
0.09329883629083004
0.09333172273294876
0.09336460917506749
0.09339749561718622
0.09343038205930494
0.09346326850142367
0.0934961549435424
0.09352904138566112
0.09356192782777985
0.09359481426989857
0.0936277007120173
0.09366058715413603
0.09369347359625475
0.09372636003837348
0.0937592464804922
0.09379213292261093
0.09382501936472966
0.09385790580684839
0.09389079224896711
0.09392367869108584
0.09395656513320456
0.09398945157532329
0.09402233801744202
0.09405522445956074
0.09408811090167947
0.0941209973437982
0.09415388378591692
0.09418677022803565
0.09421965667015438
0.0942525431122731
0.09428542955439183
0.09431831599651055
0.09435120243862928
0.09438408888074801
0.09441697532286673
0.09444986176498546
0.09448274820710419
0.09451563464922291
0.09454852109134164
0.09458

0.10651918602255804
0.10655207246467677
0.1065849589067955
0.10661784534891422
0.10665073179103295
0.10668361823315167
0.1067165046752704
0.10674939111738913
0.10678227755950785
0.10681516400162658
0.1068480504437453
0.10688093688586403
0.10691382332798276
0.10694670977010148
0.10697959621222021
0.10701248265433894
0.10704536909645766
0.10707825553857639
0.10711114198069512
0.10714402842281384
0.10717691486493257
0.1072098013070513
0.10724268774917002
0.10727557419128875
0.10730846063340747
0.1073413470755262
0.10737423351764493
0.10740711995976365
0.10744000640188238
0.1074728928440011
0.10750577928611983
0.10753866572823856
0.10757155217035728
0.10760443861247601
0.10763732505459474
0.10767021149671346
0.10770309793883219
0.10773598438095092
0.10776887082306964
0.10780175726518837
0.1078346437073071
0.10786753014942582
0.10790041659154455
0.10793330303366327
0.107966189475782
0.10799907591790073
0.10803196236001945
0.10806484880213818
0.1080977352442569
0.10813062168637563
0.10816350

0.12010128661759203
0.12013417305971076
0.12016705950182949
0.12019994594394821
0.12023283238606694
0.12026571882818567
0.12029860527030439
0.12033149171242312
0.12036437815454185
0.12039726459666057
0.1204301510387793
0.12046303748089802
0.12049592392301675
0.12052881036513548
0.1205616968072542
0.12059458324937293
0.12062746969149166
0.12066035613361038
0.12069324257572911
0.12072612901784784
0.12075901545996656
0.12079190190208529
0.12082478834420401
0.12085767478632274
0.12089056122844147
0.1209234476705602
0.12095633411267892
0.12098922055479765
0.12102210699691637
0.1210549934390351
0.12108787988115383
0.12112076632327255
0.12115365276539128
0.12118653920751
0.12121942564962873
0.12125231209174746
0.12128519853386618
0.12131808497598491
0.12135097141810364
0.12138385786022236
0.12141674430234109
0.12144963074445982
0.12148251718657854
0.12151540362869727
0.121548290070816
0.12158117651293472
0.12161406295505345
0.12164694939717217
0.1216798358392909
0.12171272228140963
0.12174560

0.1337491600968672
0.13378204653898593
0.13381493298110467
0.1338478194232234
0.13388070586534215
0.1339135923074609
0.13394647874957963
0.13397936519169837
0.1340122516338171
0.13404513807593585
0.1340780245180546
0.13411091096017333
0.13414379740229207
0.1341766838444108
0.13420957028652955
0.1342424567286483
0.13427534317076703
0.13430822961288577
0.1343411160550045
0.13437400249712325
0.134406888939242
0.13443977538136073
0.13447266182347947
0.1345055482655982
0.13453843470771695
0.1345713211498357
0.13460420759195443
0.13463709403407317
0.13466998047619191
0.13470286691831065
0.1347357533604294
0.13476863980254813
0.13480152624466687
0.13483441268678562
0.13486729912890436
0.1349001855710231
0.13493307201314184
0.13496595845526058
0.13499884489737932
0.13503173133949806
0.1350646177816168
0.13509750422373554
0.13513039066585428
0.13516327710797302
0.13519616355009176
0.1352290499922105
0.13526193643432924
0.13529482287644798
0.13532770931856672
0.13536059576068546
0.13539348220280

0.14746280646038187
0.1474956929025006
0.14752857934461935
0.1475614657867381
0.14759435222885683
0.14762723867097557
0.14766012511309431
0.14769301155521306
0.1477258979973318
0.14775878443945054
0.14779167088156928
0.14782455732368802
0.14785744376580676
0.1478903302079255
0.14792321665004424
0.14795610309216298
0.14798898953428172
0.14802187597640046
0.1480547624185192
0.14808764886063794
0.14812053530275668
0.14815342174487542
0.14818630818699416
0.1482191946291129
0.14825208107123164
0.14828496751335038
0.14831785395546912
0.14835074039758786
0.1483836268397066
0.14841651328182534
0.14844939972394408
0.14848228616606282
0.14851517260818156
0.1485480590503003
0.14858094549241904
0.14861383193453778
0.14864671837665652
0.14867960481877526
0.148712491260894
0.14874537770301274
0.14877826414513148
0.14881115058725022
0.14884403702936896
0.1488769234714877
0.14890980991360644
0.14894269635572518
0.14897558279784393
0.14900846923996267
0.1490413556820814
0.14907424212420015
0.1491071285

#### Scheme 2: (VF4 - Version 2)
The gradient in the direction normal to the edge $e_i$ is approximated by:
$$
\int_{e_i} \nabla u \cdot \mathbf{n}_{K,e_i} \, d\sigma \approx |e_i| \frac{u_{L_i} - u_K}{d_{K,e_i} + d_{L_i,e_i}}
$$
where $ K $ and $ L_i $ are the two triangles separated by the edge $ e_i $, $ d_{K,e_i} = \text{dist}(x'_K, e_i)$  and $ d_{L_i,e_i} = \text{dist}(x'_{L_i}, e_i) $. $ x'_K $ and $ x'_{L_i} $ are the centroids of triangles $ K $ and $ L_i $, respectively.


Show that the explicit FV scheme (3), using the approximation FV4 (4) for diffusion fluxes, satisfies a discrete maximum principle and is \( L^\infty \)-stable under the condition:
$$
\lambda = \max_{K \in \mathcal{T}} \left( \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K, e_{i}} + d_{L_{i}, e_{i}}} \right) \leq 1
$$
#### Maximum Principle
The explicit Euler scheme is given by:
$$
u_K^{n+1} = u_K^n - \frac{\Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} F_{K, e_{i}}^n
$$
The FV4 Version 2 scheme is given by:
$$
(FV4) \quad F_{K, e_{i}}^n = -D |e_{i}| \frac{u_{L_{i}} - u_K}{d_{K, e_{i}} + d_{L_{i}, e_{i}}}
$$
Assume $ u_K^0 \geq 0$.

Assume that $ u^n \geq 0 $ on  $\Omega$, then $ \forall K \subset \Omega, \, u_K^n \geq 0 $.

Let's show that $u_K^{n+1} \geq 0 $:

$$
u_K^{n+1} = u_K^n + \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} |e_{i}| \frac{u_{L_{i}}^n - u_K^n}{d_{K, e_{i}} + d_{L_{i}, e_{i}}}
$$
$$
= \left(1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K, e_{i}} + d_{L_{i}, e_{i}}} \right) u_K^n + \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} |e_{i}| \frac{u_{L_{i}}^n}{d_{K, e_{i}} + d_{L_{i}, e_{i}}}
$$

For $ u_K^{n+1} \geq 0 $, it suffices that:
$$
1 - \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K, e_{i}} + d_{L_{i}, e_{i}}} \geq 0 \quad \forall K
$$

And since:
$$
\lambda = \max_{K \in \mathcal{T}} \left( \frac{D \Delta t}{|K|} \sum_{e_{i} \in \epsilon_{K}} \frac{|e_{i}|}{d_{K, e_{i}} + d_{L_{i}, e_{i}}} \right) \leq 1
$$

Therefore, $ u_K^{n+1} \geq 0 \forall K $, which implies $ u_K^n \geq 0 \forall K $.

Thus, the maximum principle is verified under the condition $ \lambda \leq 1 $.


In [9]:

def centroid_triangle(Cell_id):
    A=Cells_Cord[Cell_id][1]
    B=Cells_Cord[Cell_id][2]
    C=Cells_Cord[Cell_id][0]
    x1, y1 = A
    x2, y2 = B
    x3, y3 = C
    cx = (x1 + x2 + x3) / 3
    cy = (y1 + y2 + y3) / 3
    return cx, cy

def projection_onto_line_segment(P, A, B):
    AP = np.array(P) - np.array(A)
    AB = np.array(B) - np.array(A)
    t = np.dot(AP, AB) / np.dot(AB, AB)
    t = max(0, min(1, t))
    projected_point = np.array(A) + t * AB
    return projected_point.tolist()
centroid_triangle(1)


(-1.9287757493976228, -1.215924321312566)

In [14]:

save_n=0
save_paraview_one_variable(u_init, cells._nodeid, nodes._vertex, 2, "res_VF4_v2_"+str(save_n))
temps=0
T=0.05
mx=-1
for i in range(nbcells):
    somme=0
    list_faces = cells.faceid[i]
    for k in range(3):
        mesure = faces.mesure[list_faces[k]]
        cell1=faces.cellid[list_faces[k]][0]
        cell2=faces.cellid[list_faces[k]][1]
        bar1=centroid_triangle(cell1)
        bar2=centroid_triangle(cell2)
        ar1=nodes._vertex[faces.nodeid[list_faces[k]][1]][:-2]
        ar2=nodes._vertex[faces.nodeid[list_faces[k]][0]][:-2]
        pr1=projection_onto_line_segment(bar1,ar1,ar2)
        pr2=projection_onto_line_segment(bar2,ar1,ar2)
        dst1 = dist(bar1,pr1)
        dst2= dist(bar2,pr2)
        dst=(dst1+dst2)
        somme+=mesure/dst
    K=cells.volume[i]
    somme=somme/K
    mx=max(somme,mx)
lmbda=0.8
D=1
dt1=lmbda/(D*mx) 
u_v2=np.copy(u_init)
i=0
unew = np.zeros(nbcells)
while temps<=T:
    temps+=dt1
    save_n+=1
    for j in range(nbcells):
        list_faces = cells.faceid[j]
        elem = 0
        for k in range(3):
            mesure1 = faces.mesure[list_faces[k]]
            cell1=faces.cellid[list_faces[k]][0]
            cell2=faces.cellid[list_faces[k]][1]
            bar1=centroid_triangle(cell1)
            bar2=centroid_triangle(cell2)
            ar1=nodes._vertex[faces.nodeid[list_faces[k]][1]][:-2]
            ar2=nodes._vertex[faces.nodeid[list_faces[k]][0]][:-2]
            pr1=projection_onto_line_segment(bar1,ar1,ar2)
            pr2=projection_onto_line_segment(bar2,ar1,ar2)
            dst1 = dist(bar1,pr1)
            dst2= dist(bar2,pr2)
            tmp=mesure1/(dst1+dst2)
            if j == faces.cellid[list_faces[k]][0]:
                u_diff = u_v2[faces.cellid[list_faces[k]][1]] - u_v2[faces.cellid[list_faces[k]][0]]
            else :
                u_diff = u_v2[faces.cellid[list_faces[k]][0]] - u_v2[faces.cellid[list_faces[k]][1]]
            elem += tmp*u_diff
        unew[j] = u_v2[j] - dt1/(cells.volume[j]) * (-D)*elem
    u_v2=np.copy(unew)
    i+=1
    save_paraview_one_variable(u_v2, cells._nodeid, nodes._vertex, 2, "res_VF4_v2_"+str(save_n))
    print("done",temps)

done 0.0001414875883125868
done 0.0002829751766251736
done 0.0004244627649377604
done 0.0005659503532503472
done 0.0007074379415629341
done 0.0008489255298755209
done 0.0009904131181881076
done 0.0011319007065006945
done 0.0012733882948132813
done 0.0014148758831258681
done 0.001556363471438455
done 0.0016978510597510418
done 0.0018393386480636286
done 0.0019808262363762152
done 0.002122313824688802
done 0.0022638014130013885
done 0.002405289001313975
done 0.0025467765896265617
done 0.0026882641779391483
done 0.002829751766251735
done 0.0029712393545643216
done 0.003112726942876908
done 0.003254214531189495
done 0.0033957021195020814
done 0.003537189707814668
done 0.0036786772961272547
done 0.0038201648844398413
done 0.003961652472752428
done 0.004103140061065015
done 0.004244627649377602
done 0.004386115237690189
done 0.004527602826002776
done 0.004669090414315363
done 0.00481057800262795
done 0.004952065590940537
done 0.005093553179253124
done 0.005235040767565711
done 0.005376528355

done 0.04541751584834042
done 0.045559003436653006
done 0.045700491024965595
done 0.045841978613278184
done 0.04598346620159077
done 0.04612495378990336
done 0.04626644137821595
done 0.04640792896652854
done 0.04654941655484113
done 0.046690904143153716
done 0.046832391731466305
done 0.046973879319778894
done 0.04711536690809148
done 0.04725685449640407
done 0.04739834208471666
done 0.04753982967302925
done 0.04768131726134184
done 0.04782280484965443
done 0.047964292437967015
done 0.048105780026279604
done 0.04824726761459219
done 0.04838875520290478
done 0.04853024279121737
done 0.04867173037952996
done 0.04881321796784255
done 0.04895470555615514
done 0.049096193144467726
done 0.049237680732780315
done 0.0493791683210929
done 0.04952065590940549
done 0.04966214349771808
done 0.04980363108603067
done 0.04994511867434326
done 0.05008660626265585


## Analytical Solution

Considering that the domain of calculation \( \Omega \) is unbounded, the analytical solution to the problem is:
$$
u(x, y, t) = \frac{u_{\max}}{1 + \frac{4Dt}{\sigma^2}} \exp \left( -\frac{x^2 + y^2}{\sigma^2 + 4Dt} \right)
$$

### Proof

$$
U(x,y,t) = \frac{1}{4 \pi D t} \int_{\mathbb{R}^2} \exp \left\{ - \left( \frac{(x - x')^2 + (y - y')^2}{4Dt} \right) \right\} U_0(x', y') \, dx' \, dy'
$$
$$
= \frac{1}{4 \pi D t} \int_{\mathbb{R}^2} \exp \left\{ - \left( \frac{(x - x')^2 + (y - y')^2}{4Dt} \right) \right\} U_{\text{max}} \exp \left\{ - \left( \frac{{x'}^2 + {y'}^2}{\sigma^2} \right) \right\} dx' \, dy'
$$
$$
= \frac{U_{\text{max}}}{4 \pi D t} \int_{\mathbb{R}^2} \exp \left\{ - \left( \frac{(x - x')^2}{4Dt} + \frac{{x'}^2}{\sigma^2} + \frac{(y - y')^2}{4Dt} + \frac{{y'}^2}{\sigma^2} \right) \right\}   dx' dy'
$$
$$
= \frac{U_{\text{max}}}{4 \pi D t} \exp \left\{ - \left( \frac{x^2 + y^2}{4Dt} \right) \right\} \int_{\mathbb{R}^2} \exp \left\{ - \left(    \frac{4Dt{x'}^2 + \sigma^2{x'}^2 - 2\sigma^2x x'}{4Dt \sigma^2} +  \frac{4Dt{y'}^2 + \sigma^2{y'}^2 - 2\sigma^2 y y'}{4Dt \sigma^2} \right) \right\} dx' dy'
$$

#### We have:

$$
\int_{\mathbb{R}} \exp \left\{ - \left( \frac{4Dt {x'}^2 + \sigma^2 {x'}^2 - 2\sigma^2 x{x'}}{4Dt \sigma^2} \right) \right\} dx' = \int_{\mathbb{R}} \exp \left\{ - \left( x'^2 \left( \frac{4Dt + \sigma^2}{4Dt \sigma^2} \right) - \frac{2 x x' \sigma^2}{4Dt \sigma^2} \right) \right\} dx'
$$
$$
= \int_{\mathbb{R}} \exp \left\{ - \left( \frac{4Dt + \sigma^2}{4Dt \sigma^2} \right) \left( x'^2 - \frac{2 x x' \sigma^2}{4Dt + \sigma^2} \right) \right\} dx'
$$
$$
= \int_{\mathbb{R}} \exp \left\{ -C \left( x'^2 - \frac{2 x x' \sigma^2}{4Dt + \sigma^2} \right) \right\} dx'
\quad \text{where } C = \frac{4Dt + \sigma^2}{4Dt \sigma^2}
$$
$$
= \exp \left\{C \left( \frac{x^2 \sigma^4}{(4Dt + \sigma^2)^2}\right) \right\} \int_{\mathbb{R}} \exp \left\{ -C \left( x' - \frac{x \sigma^2}{4Dt + \sigma^2} \right)^2 \right\} dx'
$$

Let $$ X = x' - \frac{x \sigma^2}{4Dt + \sigma^2} $$
then $$ \int_{\mathbb{R}} \exp \left\{ -C \left( x' - \frac{x \sigma^2}{4Dt + \sigma^2} \right)^2 \right\} dx = \int_{\mathbb{R}} \exp \left\{ -C X^2 \right\} dX = \int_{\mathbb{R}} \exp \left\{ -C x^2 \right\} dx $$

$$
= \sqrt{\frac{\pi}{C}}
$$

#### Therefore,

$$
\int_{\mathbb{R}} \exp \left\{ - \left( \frac{4Dt {x'}^2 + \sigma^2 {x'}^2 - 2\sigma^2 x{x'}}{4Dt \sigma^2} \right) \right\} dx' = \exp \left\{ C \frac{x^2 \sigma^4}{(4Dt + \sigma^2)^2} \right\} \sqrt{\frac{\pi}{C}}
$$

#### Similarly,

$$
\int_{\mathbb{R}} \exp \left\{ - \left( \frac{4Dt {y'}^2 + \sigma^2 {y'}^2 - 2\sigma^2 y{y'}}{4Dt \sigma^2} \right) \right\}dy' = \exp \left\{ C \frac{y^2 \sigma^4}{(4Dt + \sigma^2)^2} \right\} \sqrt{\frac{\pi}{C}}
$$

#### Thus,

$$
U(x, y, t) = \frac{U_{\text{max}}}{4 \pi D t} \exp \left\{ - \left( \frac{x^2 + y^2}{4Dt} \right) \right\} \int_{\mathbb{R}} \exp \left\{ - \left( \frac{4Dt {x'}^2 + \sigma^2 {x'}^2 - 2\sigma^2 x{x'}}{4Dt \sigma^2} \right) \right\} dx' \int_{\mathbb{R}} \exp \left\{ - \left( \frac{4Dt {y'}^2 + \sigma^2 {y'}^2 - 2\sigma^2 y{y'}}{4Dt \sigma^2} \right) \right\}dy'
$$
$$
= \frac{U_{\max}}{4 \pi D t} \exp\left\{ - \frac{x^2 + y^2}{4Dt} \right\} \exp \left\{ \frac{(4Dt + \sigma^2)(x^2 \sigma^4)}{(4Dt \sigma^2)(4Dt + \sigma^2)^2}\right\} \exp \left\{ \frac{(4Dt + \sigma^2)(y^2 \sigma^4)}{(4Dt \sigma^2)(4Dt + \sigma^2)^2}\right\} \frac{ 4Dt \sigma^2\pi}{4Dt + \sigma^2}
$$
$$
= \frac{U_{\max}}{1 + \frac{4Dt}{\sigma^2} }\exp \left\{ - \frac{x^2 + y^2}{4Dt} + \frac{x^2\sigma^4 + y^2\sigma^4 }{(4Dt \sigma^2)(4Dt + \sigma^2)} \right\}
$$
$$
= \frac{U_{\max}}{1 + \frac{4Dt}{\sigma^2}} \exp \left\{ \frac{-\sigma^2(4Dt + \sigma^2)(x^2+y^2) + x^2\sigma^4 + y^2\sigma^4}{(4Dt \sigma^2)(4Dt + \sigma^2)}\right\}
$$
$$
= \frac{U_{\max}}{1 + \frac{4Dt}{\sigma^2}} \exp \left\{ \frac{-(x^2+y^2)4Dt\sigma^2 }{(4Dt \sigma^2)(4Dt + \sigma^2)}\right\}
$$

Therefore,

$$
U(x, y, t) = \frac{U_{\max}}{1 + \frac{4Dt}{\sigma^2}} \exp \left\{ -\frac{x^2+y^2 }{4Dt + \sigma^2}\right\}
$$


In [17]:
def exact(temps,x,y):
    return (1/(1+(4*0.5*temps/0.25**2)))*np.exp(-1*(x**2+y**2)/((0.25**2)+4*temps*0.5))
u_exact = [ exact(0.15,cell[0], cell[1]) for cell in domain.cells.center]
u_exact = np.array(u_exact)
u_exactv2 = [ exact(0.05,cell[0], cell[1]) for cell in domain.cells.center]
u_exactv2 = np.array(u_exact)

## the L1 error between the analytical solution and the numerical solution
$$
ErrL1 = \int\!\!\!\int_{\Omega} |u(x, y, t) - \text{u_num}(x, y, t)| \, dx \, dy
= \sum_{K \in T} |K| \, |u(K) - \text{u_num}(K)|
$$


In [18]:

def norm1(exct, Sol, volume, nbelements):      
    Error = 0
    for i in range(nbelements):
        Error+= np.fabs(Sol[i]- exct[i] )* volume[i]
    return Error

In [20]:

#print(norm1(u_exact,u, cells.volume , nbcells))
print(norm1(u_exactv2,u_v2, cells.volume , nbcells))

0.04624006433565359
